### Step 0 - Generate data.csv from get_data.py

```bash
$ python get_data.py
```

In [1]:
# Import the libraries

import pandas as pd

### Step 1 - Get the data and split the features and the results

X is a matrix of features, and y is a matrix of results.

In [2]:
# Import data.csv
data = pd.read_csv('data.csv')

# Split the data into X and Y
X = data.drop(['Label', 'Source', 'CUI', 'Definition', 'Has_Definition', 'TUI'], axis=1)
Y = data['TUI_Index']

print("Features:\n", X.head())
print("Results:\n", Y.head())

Features:
    Nb_Parents  Nb_Children  Has_Nb_Parents_And_Children  TUI_Index
0         0.9          0.9                            1         98
1         0.9          0.9                            1        100
2         0.9          0.9                            1        107
3         1.0          1.0                            1         96
4         1.0          1.0                            1        100
Results:
 0     98
1    100
2    107
3     96
4    100
Name: TUI_Index, dtype: int64
